In [1]:
import os
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# trainig set과 test set의 모든 이미지 앞이 대해서,
# jpg image header가 포함되지 않은 (jpg의 파일 구조에 어긋나는) 파일들을 삭제

data_path = '/aiffel/aiffel/model-fit/data/30vnfoods/'
train_path = data_path + 'Train/'
test_path = data_path + 'Test/'

for path in [train_path, test_path]:
    classes = os.listdir(path)
    
    for food in classes:
        food_path = os.path.join(path, food)
        images = os.listdir(food_path)
        
        for image in images:
            with open(os.path.join(food_path, image), 'rb') as f:
                bytes = f.read()
            if bytes[:3] != b'\xff\xd8\xff':
                print(os.path.join(food_path, image))
                os.remove(os.path.join(food_path, image))

In [3]:
# 개수 확인
classes = os.listdir(train_path)
train_length = 0

for food in classes:
    food_path = os.path.join(train_path, food)
    images = os.listdir(food_path)
    
    train_length += len(images)
    
print('trainig data의 개수: ' +str(train_length))

trainig data의 개수: 9775


In [4]:
# 문제1: dataloader 구현하기

def process_path(file_path, class_names, img_shape=(224, 224)):
    '''
    file_path로부터 class label을 만들고, 이미지를 읽는 함수
    '''
    label = tf.strings.split(file_path, os.path.sep)
    label = label[-2] == class_names

    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, img_shape)
    return img, label

def prepare_for_training(ds, batch_size=32, cache=True, shuffle_buffer_size=1000):
    '''
    TensorFlow Data API를 이용해 data batch를 만드는 함수
    '''
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat(5)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

def load_data(data_path, batch_size=32):
    '''
    데이터를 만들기 위해 필요한 함수들을 호출하고 데이터를 리턴해주는 함수
    '''
    class_names = [cls for cls in os.listdir(data_path) if cls != '.DS_Store']
    data_path = pathlib.Path(data_path)

    list_ds = tf.data.Dataset.list_files(str(data_path/'*/*'))
    labeled_ds = list_ds.map(lambda x: process_path(x, class_names, img_shape=(224, 224)))
    ds = prepare_for_training(labeled_ds, batch_size=batch_size)

    return ds

In [5]:
# 문제2: 모델 구현하기

from tensorflow.keras.applications import EfficientNetB0

class Model(tf.keras.Model):
    '''
    EfficientNetB0을 백본으로 사용하는 모델을 구성합니다.
    Classification 문제로 접근할 것이기 때문에 맨 마지막 Dense 레이어에 
    우리가 원하는 클래스 개수만큼을 지정해주어야 합니다.
    '''
    def __init__(self, num_classes=10, freeze=False):
        super(Model, self).__init__()
        self.base_model = EfficientNetB0(include_top=False, weights='imagenet')
        if freeze:
            self.base_model.trainable = False
        self.top = tf.keras.Sequential([tf.keras.layers.GlobalAveragePooling2D(name="avg_pool"),
                                       tf.keras.layers.BatchNormalization(),
                                       tf.keras.layers.Dropout(0.5, name="top_dropout")])
        self.classifier = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")

    def call(self, inputs, training=True):
        # [[YOUR CODE]]
        x = self.base_model(inputs)
        x = self.top(x)
        x = self.classifier(x)
        return x
    
if __name__ =='__main__':
    model = Model(num_classes=10, freeze=True)
    model.build(input_shape=(None, 224, 224, 3))
    print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
sequential (Sequential)      (None, 1280)              5120      
_________________________________________________________________
pred (Dense)                 multiple                  12810     
Total params: 4,067,501
Trainable params: 15,370
Non-trainable params: 4,052,131
_________________________________________________________________
None


In [6]:
# 문제3: custom trainer 구현하기

class Trainer:
    def __init__(self, model,ds_length, epochs, batch, loss_fn, optimizer):
        self.model = model
        self.epochs = epochs
        self.batch = batch
        self.loss_fn = loss_fn
        self.optimizer = optimizer
    def train(self, train_dataset, train_metric):
        for epoch in range(self.epochs):
            print("\nStart of epoch %d" % (epoch+1,))
            # [[YOUR CODE]]
            for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
                with tf.GradientTape() as tape:
                    logits = model(x_batch_train)
                    loss_value = self.loss_fn(y_batch_train, logits)
                grads = tape.gradient(loss_value, model.trainable_weights)
                self.optimizer.apply_gradients(zip(grads, model.trainable_weights))
                # train metric 업데이트
                train_metric.update_state(y_batch_train, logits)
                # 5 배치마다 로깅
                if step % 5 == 0:
                    print(
                        "Training loss (for one batch) at step %d: %.4f"
                        % (step, float(loss_value))
                    )
                    print("Seen so far: %d samples" % ((step + 1) * self.batch))
                    print(train_metric.result().numpy())
            # 마지막 epoch 학습이 끝나면 train 결과를 보여줌
            train_acc = train_acc_metric.result()
            print("Training acc over epoch: %.4f" % (float(train_acc),))

In [7]:
# 모델 학습 코드

train_path = "/aiffel/aiffel/model-fit/data/30vnfoods/Train"

epoch = 10
batch = 64

model = Model(num_classes=10)
dataset = load_data(data_path=train_path, batch_size=batch)
loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
trainer = Trainer(model=model,
                epochs=epoch,
                batch=batch,
                ds_length=train_length,
                loss_fn=loss_function,
                optimizer=optimizer)

trainer.train(train_dataset=dataset,
            train_metric=train_acc_metric)


Start of epoch 1
Training loss (for one batch) at step 0: 4.0665
Seen so far: 64 samples
0.0625
Training loss (for one batch) at step 5: 2.5827
Seen so far: 384 samples
0.22395833
Training loss (for one batch) at step 10: 1.8439
Seen so far: 704 samples
0.3224432
Training loss (for one batch) at step 15: 1.4730
Seen so far: 1024 samples
0.40527344
Training loss (for one batch) at step 20: 1.1852
Seen so far: 1344 samples
0.45386904
Training loss (for one batch) at step 25: 1.1217
Seen so far: 1664 samples
0.49278846
Training loss (for one batch) at step 30: 0.9645
Seen so far: 1984 samples
0.5267137
Training loss (for one batch) at step 35: 0.6311
Seen so far: 2304 samples
0.55772567
Training loss (for one batch) at step 40: 0.8912
Seen so far: 2624 samples
0.5788872
Training loss (for one batch) at step 45: 0.6177
Seen so far: 2944 samples
0.59748644
Training loss (for one batch) at step 50: 0.7697
Seen so far: 3264 samples
0.6136642
Training loss (for one batch) at step 55: 0.5843
S

Training loss (for one batch) at step 475: 0.2608
Seen so far: 30464 samples
0.86580884
Training loss (for one batch) at step 480: 0.1719
Seen so far: 30784 samples
0.8664566
Training loss (for one batch) at step 485: 0.0671
Seen so far: 31104 samples
0.86741257
Training loss (for one batch) at step 490: 0.3120
Seen so far: 31424 samples
0.86799896
Training loss (for one batch) at step 495: 0.1690
Seen so far: 31744 samples
0.8687941
Training loss (for one batch) at step 500: 0.1823
Seen so far: 32064 samples
0.8696981
Training loss (for one batch) at step 505: 0.2542
Seen so far: 32384 samples
0.8703063
Training loss (for one batch) at step 510: 0.0630
Seen so far: 32704 samples
0.87102497
Training loss (for one batch) at step 515: 0.0843
Seen so far: 33024 samples
0.8718508
Training loss (for one batch) at step 520: 0.1437
Seen so far: 33344 samples
0.8725108
Training loss (for one batch) at step 525: 0.0751
Seen so far: 33664 samples
0.8733068
Training loss (for one batch) at step 5

Training loss (for one batch) at step 180: 0.1469
Seen so far: 11584 samples
0.9120396
Training loss (for one batch) at step 185: 0.0496
Seen so far: 11904 samples
0.9123217
Training loss (for one batch) at step 190: 0.0623
Seen so far: 12224 samples
0.91265
Training loss (for one batch) at step 195: 0.0531
Seen so far: 12544 samples
0.91302365
Training loss (for one batch) at step 200: 0.0280
Seen so far: 12864 samples
0.9133773
Training loss (for one batch) at step 205: 0.0724
Seen so far: 13184 samples
0.91371113
Training loss (for one batch) at step 210: 0.2110
Seen so far: 13504 samples
0.9139935
Training loss (for one batch) at step 215: 0.0347
Seen so far: 13824 samples
0.9143208
Training loss (for one batch) at step 220: 0.0793
Seen so far: 14144 samples
0.9145972
Training loss (for one batch) at step 225: 0.0080
Seen so far: 14464 samples
0.9149497
Training loss (for one batch) at step 230: 0.1451
Seen so far: 14784 samples
0.91520447
Training loss (for one batch) at step 235:

Training loss (for one batch) at step 650: 0.0195
Seen so far: 41664 samples
0.9313224
Training loss (for one batch) at step 655: 0.1153
Seen so far: 41984 samples
0.93153125
Training loss (for one batch) at step 660: 0.1624
Seen so far: 42304 samples
0.93161803
Training loss (for one batch) at step 665: 0.2814
Seen so far: 42624 samples
0.9317697
Training loss (for one batch) at step 670: 0.0471
Seen so far: 42944 samples
0.9319313
Training loss (for one batch) at step 675: 0.0425
Seen so far: 43264 samples
0.9320483
Training loss (for one batch) at step 680: 0.1288
Seen so far: 43584 samples
0.9321537
Training loss (for one batch) at step 685: 0.0101
Seen so far: 43904 samples
0.93230146
Training loss (for one batch) at step 690: 0.0129
Seen so far: 44224 samples
0.9324697
Training loss (for one batch) at step 695: 0.1276
Seen so far: 44544 samples
0.93260473
Training loss (for one batch) at step 700: 0.2718
Seen so far: 44864 samples
0.93271744
Training loss (for one batch) at step 

Training loss (for one batch) at step 355: 0.0136
Seen so far: 22784 samples
0.9433106
Training loss (for one batch) at step 360: 0.1124
Seen so far: 23104 samples
0.9433697
Training loss (for one batch) at step 365: 0.0169
Seen so far: 23424 samples
0.9434615
Training loss (for one batch) at step 370: 0.0187
Seen so far: 23744 samples
0.94356924
Training loss (for one batch) at step 375: 0.1118
Seen so far: 24064 samples
0.9436354
Training loss (for one batch) at step 380: 0.0545
Seen so far: 24384 samples
0.9437421
Training loss (for one batch) at step 385: 0.0609
Seen so far: 24704 samples
0.94376665
Training loss (for one batch) at step 390: 0.1262
Seen so far: 25024 samples
0.9438481
Training loss (for one batch) at step 395: 0.0319
Seen so far: 25344 samples
0.9439372
Training loss (for one batch) at step 400: 0.0726
Seen so far: 25664 samples
0.944042
Training loss (for one batch) at step 405: 0.0056
Seen so far: 25984 samples
0.9441221
Training loss (for one batch) at step 410:

...

Training loss (for one batch) at step 295: 0.0609
Seen so far: 18944 samples
0.9691638
Training loss (for one batch) at step 300: 0.0087
Seen so far: 19264 samples
0.96917725
Training loss (for one batch) at step 305: 0.0237
Seen so far: 19584 samples
0.96917963
Training loss (for one batch) at step 310: 0.0606
Seen so far: 19904 samples
0.96919584
Training loss (for one batch) at step 315: 0.0066
Seen so far: 20224 samples
0.9692065
Training loss (for one batch) at step 320: 0.0718
Seen so far: 20544 samples
0.96921986
Training loss (for one batch) at step 325: 0.0512
Seen so far: 20864 samples
0.9692415
Training loss (for one batch) at step 330: 0.1156
Seen so far: 21184 samples
0.9692493
Training loss (for one batch) at step 335: 0.0418
Seen so far: 21504 samples
0.9692654
Training loss (for one batch) at step 340: 0.0466
Seen so far: 21824 samples
0.9692759
Training loss (for one batch) at step 345: 0.0050
Seen so far: 22144 samples
0.9692974
Training loss (for one batch) at step 3

Training acc over epoch: 0.9706

Start of epoch 9
Training loss (for one batch) at step 0: 0.0035
Seen so far: 64 samples
0.9706186
Training loss (for one batch) at step 5: 0.0004
Seen so far: 384 samples
0.97064006
Training loss (for one batch) at step 10: 0.0033
Seen so far: 704 samples
0.97065896
Training loss (for one batch) at step 15: 0.0008
Seen so far: 1024 samples
0.9706778
Training loss (for one batch) at step 20: 0.0012
Seen so far: 1344 samples
0.9706992
Training loss (for one batch) at step 25: 0.0008
Seen so far: 1664 samples
0.9707205
Training loss (for one batch) at step 30: 0.0123
Seen so far: 1984 samples
0.97073674
Training loss (for one batch) at step 35: 0.0107
Seen so far: 2304 samples
0.97075796
Training loss (for one batch) at step 40: 0.0077
Seen so far: 2624 samples
0.9707716
Training loss (for one batch) at step 45: 0.0016
Seen so far: 2944 samples
0.97079027
Training loss (for one batch) at step 50: 0.0408
Seen so far: 3264 samples
0.9708089
Training loss (f

Training loss (for one batch) at step 470: 0.0050
Seen so far: 30144 samples
0.97189325
Training loss (for one batch) at step 475: 0.1107
Seen so far: 30464 samples
0.97190505
Training loss (for one batch) at step 480: 0.0038
Seen so far: 30784 samples
0.9719193
Training loss (for one batch) at step 485: 0.0445
Seen so far: 31104 samples
0.9719335
Training loss (for one batch) at step 490: 0.1314
Seen so far: 31424 samples
0.9719524
Training loss (for one batch) at step 495: 0.0141
Seen so far: 31744 samples
0.9719665
Training loss (for one batch) at step 500: 0.0018
Seen so far: 32064 samples
0.97197115
Training loss (for one batch) at step 505: 0.0389
Seen so far: 32384 samples
0.9719758
Training loss (for one batch) at step 510: 0.0337
Seen so far: 32704 samples
0.97199225
Training loss (for one batch) at step 515: 0.0694
Seen so far: 33024 samples
0.97200394
Training loss (for one batch) at step 520: 0.0082
Seen so far: 33344 samples
0.9720133
Training loss (for one batch) at step 

Training loss (for one batch) at step 175: 0.0019
Seen so far: 11264 samples
0.97291523
Training loss (for one batch) at step 180: 0.1556
Seen so far: 11584 samples
0.97292334
Training loss (for one batch) at step 185: 0.0002
Seen so far: 11904 samples
0.9729403
Training loss (for one batch) at step 190: 0.0094
Seen so far: 12224 samples
0.97295284
Training loss (for one batch) at step 195: 0.0112
Seen so far: 12544 samples
0.97296757
Training loss (for one batch) at step 200: 0.0996
Seen so far: 12864 samples
0.9729778
Training loss (for one batch) at step 205: 0.0164
Seen so far: 13184 samples
0.97298366
Training loss (for one batch) at step 210: 0.0903
Seen so far: 13504 samples
0.9729983
Training loss (for one batch) at step 215: 0.0862
Seen so far: 13824 samples
0.9730107
Training loss (for one batch) at step 220: 0.0010
Seen so far: 14144 samples
0.9730232
Training loss (for one batch) at step 225: 0.0072
Seen so far: 14464 samples
0.9730378
Training loss (for one batch) at step 

Training loss (for one batch) at step 645: 0.0974
Seen so far: 41344 samples
0.9738975
Training loss (for one batch) at step 650: 0.1526
Seen so far: 41664 samples
0.9739024
Training loss (for one batch) at step 655: 0.0030
Seen so far: 41984 samples
0.97391975
Training loss (for one batch) at step 660: 0.2030
Seen so far: 42304 samples
0.97393084
Training loss (for one batch) at step 665: 0.0196
Seen so far: 42624 samples
0.97393984
Training loss (for one batch) at step 670: 0.1173
Seen so far: 42944 samples
0.9739364
Training loss (for one batch) at step 675: 0.0068
Seen so far: 43264 samples
0.97394747
Training loss (for one batch) at step 680: 0.1078
Seen so far: 43584 samples
0.9739482
Training loss (for one batch) at step 685: 0.0317
Seen so far: 43904 samples
0.9739571
Training loss (for one batch) at step 690: 0.0222
Seen so far: 44224 samples
0.97395575
Training loss (for one batch) at step 695: 0.0751
Seen so far: 44544 samples
0.9739606
Training loss (for one batch) at step 

In [8]:
# 모델 테스트 코드

test_ds = load_data(data_path=test_path)

for step_train, (x_batch_train, y_batch_train) in enumerate(test_ds.take(10)):
    prediction = model(x_batch_train)
    print("{}/{}".format(np.array(tf.equal(tf.argmax(y_batch_train, axis=1), tf.argmax(prediction, axis=1))).sum(), tf.argmax(y_batch_train, axis=1).shape[0]))

27/32
26/32
28/32
30/32
27/32
29/32
30/32
27/32
28/32
27/32
